# Topological feature extraction from (un)directed graphs: `VietorisRipsPersistence`, `SparseRipsPersistence` and `FlagserPersistence`

`giotto-tda` allows the extraction of topological features from undirected or directed graphs represented as adjacency matrices, via the following transformers:

- [`VietorisRipsPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) and [`SparseRipsPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) initialized with `metric="precomputed"`, for undirected graphs;
- [`FlagserPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) initialized with `directed=True`, for directed graphs (and with `directed=False` for undirected ones).

In this notebook, we build some basic intuition on how these methods are able to capture structures and patterns from such graphs. We will focus on the multi-scale nature of the information contained in the final outputs ("persistence diagrams"), as well as on the differences between the undirected and directed cases. Another important point is that, although adjacency matrices of sparsely connected and even unweighted graphs can be passed directly to these transformers, they will always be interpreted as **weighted adjacency matrices** according to some not completely standard conventions. We will highlight these below.

The mathematical technologies used under the hood are various flavours of "persistent homology" (as is also the case for [`EuclideanCechPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.EuclideanCechPersistence.html) and [`CubicalPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.CubicalPersistence.html)). If you are interested in the details, you can start from the [theory glossary](https://giotto-ai.github.io/gtda-docs/latest/theory/glossary.html) and references therein.

If you are looking at a static version of this notebook and would like to run its contents, head over to [github](https://github.com/giotto-ai/giotto-tda/blob/master/examples/persistent_homology_graphs.ipynb).


## See also

- [Topological feature extraction using `VietorisRipsPersistence` and `PersistenceEntropy`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/vietoris_rips_quickstart.html) which treats the "special case" of point clouds (see below).
- [Plotting in `giotto-tda`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/plotting_api.html), particularly Section 1.2 (as above, treats the case of point clouds).
- [Case study: Classification of shapes](https://giotto-ai.github.io/gtda-docs/latest/notebooks/classifying_shapes.html) (a more advanced example).

**License: AGPLv3**

## Import libraries

In [1]:
import numpy as np
from numpy.random import default_rng
rng = default_rng(42)  # Create a random number generator

from scipy.sparse import csr_matrix

from gtda.graphs import GraphGeodesicDistance
from gtda.homology import VietorisRipsPersistence, SparseRipsPersistence, FlagserPersistence

from igraph import Graph

from os import mkdir
from IPython.display import SVG, display

## Undirected graphs – `VietorisRipsPersistence` and `SparseRipsPersistence`

### Fully connected and weighted

We begin with the case of fully connected and weighted (FCW) undirected graphs. The short version of the story is that if you have a collection `X` of weighted adjacency matrices (which could be sparse or dense, and possibly upper-triangular), you can instantiate tranformers of class `VietorisRipsPersistence` or `SparseRipsPersistence` by setting the parameter `metric` as `"precomputed"`, and then call `fit_transform` on `X` to obtain the corresponding collection of *persistence diagrams*.

In [2]:
# Start creating a single 10 x 10 weighted adjacency matrix (of a FCW graph)
x = rng.random((10, 10))
# Fill the diagonal with zeros (not always necessary, see below)
np.fill_diagonal(x, 0)

# Create a trivial collection of weighted adjacency matrices, containing x only
X = [x]  # Alternative as a 3D array: X = x.reshape((1, **x.shape))

# Instantiate transformers with `metric="precomputed"` and default remaining parameters
VR, SR = VietorisRipsPersistence(metric="precomputed"), SparseRipsPersistence(metric="precomputed", epsilon=0.1)

# Compute persistence diagrams corresponding to each entry in X
X_vr, X_sr = VR.fit_transform(X), SR.fit_transform(X)

print(f"X_vr.shape: {X_vr.shape} (16 topological features)\nX_sr.shape: {X_sr.shape} (15 topological features)")

X_vr.shape: (1, 15, 3) (16 topological features)
X_sr.shape: (1, 14, 3) (15 topological features)


`SparseRipsPersistence` actually implements an approximate scheme for computing the same topological quantities as `VietorisRipsPersistence`, with possibly very large speedups when the inputs have large sizes. We can plot the two results:

In [4]:
sample = 0
for transformer, output in [(VR, X_vr), (SR, X_sr)]:
    title = f"Persistence diagram computed from FCW graph {sample}<br>using {transformer.__class__.__name__}"
    VR.plot(output,
            sample=sample,
            plotly_params={"layout": {"title": title}}).show()

... Not as similar as we might have hoped! This is to be expected with such a small input matrix and with the default value of `SparseRipsPersistence`'s `epsilon` parameter. But instead of trying to change things, we will leave `SparseRipsPersistence` aside for the rest of the notebook and focus on the meaning of the output in the case of `VietorisRipsPersistence`.

### Understanding the computation 

To understand what these persistence diagrams are telling us about the input FCW graphs, we briefly explain the "clique complex (or "flag complex") filtration" procedure underlying the computations in `VietorisRipsPersistence` when `metric="precomputed"`, via an example.

Let us start with a special case of a FCW graph with adjacency matrix as follows:
- the diagonal entries ("node weights") are all zero;
- all off-diagonal entries (edge weights) are non-negative;
- some edge weights are infinite.

We can lay such a graph on the plane to visualise it, drawing only the finite edges:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/graph.png)
The procedure then can be explained as follows: we let a parameter $\varepsilon$ start at $0$, and as we increase it we keep asking the following questions:
1. What are the current connected components if we consider all vertices, but disregard all edges whose weight is strictly greater than the current value of $\varepsilon$?
2. Considering the same edges as above, for each integer $k \geq 1$, what new $(k + 1)$-cliques have appeared? These are also called *$k$-simplices* in this context. $1$-simplices are edges, $2$-simplices are "triangles", $3$-simplices are "tetrahedra", etc, but *note* that despite the geometric terminology, these objects are purely abstract/combinatorial.
3. Have any new 1-dimensional "loops", 2-dimensional "cavities", and more generally $d$-dimensional voids appeared which were not present at earlier values of $\epsilon$? A loop, cavity, or $d$-dimensional void is such only if there is no collection of "triangles", "tetrahedra", or $(d + 1)$-simplices which collectively "bound" it. For instance, although the edges of a triangle together "go in a loop", in this construction a triangle will also be declared present when all its edges are, making its edges not eligible to true void status.
4. Conversely, have any of the $d$-dimensional voids which were present at earlier $\epsilon$ been "filled" by new $(d + 1)$-simplices which have just appeared?

Let us start at $\varepsilon = 0$: Some edges had zero weight in our graph, so they already appear!
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_0.png)
There are 10 connected components, and nothing much else.

Up to and including $\varepsilon = 2$, a few more edges are added which make some of the connected components merge together but do not create any loops, triangles, or higher cliques. Let us look at $\varepsilon = 3$:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_2.png)
The newly arrived edges reduce the number of connected components further, but more interestingly create two triangles.

Up to and including $\varepsilon = 6$, more connected components are merged by edges and more triangles appear. At $\varepsilon = 7$, a tetrahedron appears:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_6.png)

At $\varepsilon = 8$, our first one-dimensional loop appears (the fact that some edges can intersect in the graph representation is an artifact of laying out the graph on the plane, and is not relevant to the "loop status"):
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_7.png)

At $\varepsilon = 9$, our first one-dimensional loop appears (the fact that some edges can intersect in the graph representation is an artifact of laying out the graph on the plane, and is not relevant to the "loop status"):
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_8.png)

### The "special case" of point clouds

#### Advanced discussion: Node weights



In [ ]:
from plotly import graph_objects as go
N = 13
pos = rng.random((2, N))

In [ ]:
n_edges = 100
edges_source = rng.choice(np.arange(n_edges), replace=False, size=n_edges) % N
edges_target = rng.choice(np.arange(n_edges), replace=False, size=n_edges) % N
edges = np.unique(np.sort(np.vstack([edges_source, edges_target]), axis=0), axis=1).T
edges = edges[edges[:, 0] != edges[:, 1]]
x_edges_source = pos[0][edges[:, 0]]
x_edges_target = pos[0][edges[:, 1]]
y_edges_source = pos[1][edges[:, 0]]
y_edges_target = pos[1][edges[:, 1]]
list_x_edges = []
list_x_midpts = []
list_y_edges = []
list_y_midpts = []
weights_final = []
idx_final = []
list_of_i = []
for i, edge in enumerate(edges):
    source_idx, target_idx = edge
    if np.linalg.norm(pos[:, source_idx] - pos[:, target_idx]) < 0.4:
        weights_final.append(rng.integers(0, 10))
        idx_final += [source_idx, target_idx]
        list_x_edges += [None, x_edges_source[i], x_edges_target[i]]
        list_x_midpts.append((x_edges_source[i] + x_edges_target[i]) / 2)
        list_y_edges += [None, y_edges_source[i], y_edges_target[i]]
        list_y_midpts.append((y_edges_source[i] + y_edges_target[i]) / 2)
        list_of_i.append(i)
unique_idx_final = np.unique(idx_final)
unique_idx_final_sorted = np.sort(unique_idx_final)
weights_final = np.asarray(weights_final)

comm_layout = go.Layout(
    plot_bgcolor="white",
    height=700, width=700,
    xaxis={'visible': False, "range": [0, 1]},
    yaxis={'visible': False, "range": [0, 1]},
    showlegend=False
)
figs = []
fig = go.Figure(layout=comm_layout)
fig.add_traces(go.Scatter(x=list_x_edges, y=list_y_edges, mode='lines', line_color='red'))
fig.add_traces(go.Scatter(x=pos[0, unique_idx_final_sorted], y =pos[1, unique_idx_final_sorted], mode='markers', marker_color='blue', marker_size=10))
fig.add_traces(go.Scatter(x=list_x_midpts, y=list_y_midpts, 
                          mode='text',
                          text=weights_final,
                          textfont={"size": 15},
                          textposition='bottom center',
                          hoverinfo='text'))
figs.append(fig)

for w in range(10):
    relevant_weights = np.flatnonzero(weights_final <= w)
    list_x_edges_temp = []
    list_y_edges_temp = []
    for j in relevant_weights:
        list_x_edges_temp += list_x_edges[3 * j: 3 * (j + 1)]
        list_y_edges_temp += list_y_edges[3 * j: 3 * (j + 1)]
    fig = go.Figure(layout=comm_layout)
    fig.add_traces(go.Scatter(x=list_x_edges_temp, y=list_y_edges_temp, mode='lines', line_color='red'))
    edges[list_of_i][relevant_weights]
    unique_idx_final_temp_sorted = np.sort(np.unique(edges[list_of_i][relevant_weights]))
    fig.add_traces(go.Scatter(x=pos[0, unique_idx_final_sorted], y =pos[1, unique_idx_final_sorted], mode='markers', marker_color='blue', marker_size=10))
    fig.add_traces(go.Scatter(x=np.asarray(list_x_midpts)[relevant_weights], y=np.asarray(list_y_midpts)[relevant_weights], 
                              mode='text',
                              text=weights_final[relevant_weights],
                              textfont={"size": 15},
                              textposition='bottom center',
                              hoverinfo='text'))
    figs.append(fig)
figs[0]

### The unweighted case: Chaining with `GraphGeodesicDistance`

But what if, as is the case in many applications, our data samples come in the form not of FCW graphs, but of graphs with much sparser connections, and potentially no edge weights? `VietorisRipsPersistence` and `SparseRipsPersistence` 

By preprocessing via [`GraphGeodesicDistance`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/graphs/processing/gtda.graphs.GraphGeodesicDistance.html) to obtain weighted graphs from unweighted ones, (directed or undirected) **unweighted** graphs are also supported in pipelines.

## Directed graphs and `FlagserPersistence`

Together with the companion package `pyflagser` ([source code](https://github.com/giotto-ai/pyflagser), [API reference](https://docs-pyflagser.giotto.ai/)), `giotto-tda` allows the extraction of topological features from *directed* (weighted or unweighted) graphs. 

In [ ]:
def make_directed_circle_adjacency(n_vertices):
    weights = np.ones(n_vertices)
    rows = np.arange(n_vertices)
    columns = np.arange(1, n_vertices + 1) % n_vertices
    return csr_matrix((weights, (rows, columns)))

n_vertices = 10
directed_circle = make_directed_circle_adjacency(n_vertices)

In [ ]:
homology_dimensions = [0, 1, 2]

In [ ]:
row, col = directed_circle.nonzero()
graph = Graph(n=n_vertices, edges=list(zip(row, col)), directed=True)
# plot(graph)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle])
undirected_circle_gd = GraphGeodesicDistance(directed=False).fit_transform([directed_circle])

In [ ]:
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_gd);

In [ ]:
FP = FlagserPersistence(directed=False, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(undirected_circle_gd);

# # Alternative
# VR = VietorisRipsPersistence(metric="precomputed", homology_dimensions=homology_dimensions)
# VR.fit_transform_plot(undirected_circle_gd);

In [ ]:
rows_flipped = np.concatenate([row[::2], col[1::2]])
columns_flipped = np.concatenate([col[::2], row[1::2]])
weights = np.ones(n_vertices)
directed_circle_flipped = csr_matrix((weights, (rows_flipped, columns_flipped)), shape=(n_vertices, n_vertices))

graph = Graph(n=n_vertices, edges=list(zip(rows_flipped, columns_flipped)), directed=True)
# plot(graph)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_flipped_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle_flipped])
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_flipped_gd);

In [ ]:
rows_fixed_point = np.concatenate([row[:n_vertices // 2], col[n_vertices // 2:]])
columns_fixed_point = np.concatenate([col[:n_vertices // 2], row[n_vertices // 2:]])
weights = np.ones(n_vertices)
directed_circle_fixed_point = csr_matrix((weights, (rows_fixed_point, columns_fixed_point)), shape=(n_vertices, n_vertices))

graph = Graph(n=n_vertices, edges=list(zip(rows_fixed_point, columns_fixed_point)), directed=True)
# plot(graph)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_fixed_point_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle_fixed_point])
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_fixed_point_gd);

# Figure-eight experiments

In [ ]:
N = 10
M = 20

fig_8 = np.full((N + M - 1, N + M - 1), fill_value=np.inf)
fig_8[range(N - 1), range(1, N)] = 1
fig_8[N - 1, 0] = 1
fig_8[range(N - 1, N + M - 2), range(N, N + M - 1)] = 1
fig_8[N + M - 2, N - 1] = 1
np.fill_diagonal(fig_8, 0)

graph = Graph.Adjacency(fig_8.tolist())
plot(graph)

In [ ]:
fig_8_gd = GraphGeodesicDistance(directed=True).fit_transform([fig_8])
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(fig_8_gd);

## Undirected case

In [ ]:
N = 10
M = 20

fig_8 = np.full((N + M - 1, N + M - 1), fill_value=0)
fig_8[range(N - 1), range(1, N)] = 1
fig_8[N - 1, 0] = 1
fig_8[range(N - 1, N + M - 2), range(N, N + M - 1)] = 1
fig_8[N + M - 2, N - 1] = 1

fig_8 = fig_8 + fig_8.T
fig_8 = np.where(fig_8, fig_8 != 0, np.inf)

np.fill_diagonal(fig_8, 0)

graph = Graph.Adjacency(fig_8.tolist())
plot(graph)

In [ ]:
fig_8_gd = GraphGeodesicDistance(directed=False).fit_transform([fig_8])
VR = VietorisRipsPersistence(metric='precomputed', homology_dimensions=homology_dimensions)
VR.fit_transform_plot(fig_8_gd);